In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score

In [ ]:
import pandas as pd

# Cargar el conjunto de datos desde un archivo CSV
data_path = r"C:\Proyectos programación\Modelos\consumo_casa.csv"
data = pd.read_csv(data_path)

# Eliminar la primera columna (asumiendo que se llama 'Unnamed: 0')
data.drop(columns=['Unnamed: 0'], inplace=True)

# Calcular la columna 'otros'
data['otros'] = data['Medidor [W]'] - data[['Refrigerator', 'Clothes washer', 'Clothes Iron', 'Computer', 'Oven', 'Play', 'TV', 'Sound system']].sum(axis=1)

# Convertir la columna 'Fecha' a tipo datetime si no lo está
data['Fecha'] = pd.to_datetime(data['Fecha'])

print(data)


In [ ]:
# Dividir los datos en características (X) y etiquetas (y)
X = data[['Fecha', 'Medidor [W]']]
y = data.drop(columns=['Fecha', 'Medidor [W]'])

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [71]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Aplicar funciones seno y coseno a las variables de predicción (y)
variables_prediccion = ['Refrigerator', 'Clothes washer', 'Clothes Iron', 'Computer', 'Oven', 'Play', 'TV', 'Sound system','Medidor [W]']
for variable in variables_prediccion:
    data[f'{variable}_seno'] = np.sin(2 * np.pi * data[variable] / 24)
    data[f'{variable}_coseno'] = np.cos(2 * np.pi * data[variable] / 24)

# Dividir los datos en características (X) y etiquetas (y)
X = data[['mes_seno', 'mes_coseno', 'dia_seno', 'dia_coseno', 'hora_seno', 'hora_coseno', 'minuto_seno', 'minuto_coseno', 'Medidor [W]_seno','Medidor [W]_coseno']]
y = data[[f'{variable}_seno' for variable in variables_prediccion] + [f'{variable}_coseno' for variable in variables_prediccion]]

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Construir el modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluar el modelo
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print("R2 score en entrenamiento:", train_score)
print("R2 score en prueba:", test_score)


R2 score en entrenamiento: 0.13983053749368263
R2 score en prueba: 0.13933064768228232


In [72]:
from sklearn.metrics import mean_squared_error

# Predecir los valores en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)

print("Error cuadrático medio:", mse)


Error cuadrático medio: 0.09793325161737038


In [73]:
from sklearn.metrics import mean_squared_error

# Predecir los valores en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio por cada columna
mse_por_columna = {}
for i, columna in enumerate(y.columns):
    mse_por_columna[columna] = mean_squared_error(y_test.iloc[:, i], y_pred[:, i])

# Imprimir el MSE por cada columna
for columna, mse_valor in mse_por_columna.items():
    print(f"MSE para {columna}: {mse_valor}")


MSE para Refrigerator_seno: 0.19686138860436575
MSE para Clothes washer_seno: 0.03727592423966446
MSE para Clothes Iron_seno: 0.014280187090053235
MSE para Computer_seno: 0.030201722579856895
MSE para Oven_seno: 0.016553057909056493
MSE para Play_seno: 0.08538070645744361
MSE para TV_seno: 0.12280961974231075
MSE para Sound system_seno: 0.1407520559965206
MSE para Medidor [W]_seno: 1.5990595699995692e-30
MSE para Refrigerator_coseno: 0.6144699484221425
MSE para Clothes washer_coseno: 0.07121330715563971
MSE para Clothes Iron_coseno: 0.008075897520595685
MSE para Computer_coseno: 0.0387663220297995
MSE para Oven_coseno: 0.005708769807957872
MSE para Play_coseno: 0.03266746266276407
MSE para TV_coseno: 0.22825347401870738
MSE para Sound system_coseno: 0.11952868487578508
MSE para Medidor [W]_coseno: 5.064937891975116e-31


In [74]:
# Elegir la variable para la cual mostrar los valores (por ejemplo, 'Refrigerator')
variable_elegida = 'Refrigerator'

# Obtener los primeros 5 valores reales y predichos para la variable elegida
primeros_5_valores = pd.DataFrame({
    'Valor Real': y_test[f'{variable_elegida}_seno'].iloc[:10].values,
    'Valor Predicho': y_pred[:10, y.columns.get_loc(f'{variable_elegida}_seno')]
})

# Mostrar los primeros 5 valores reales y predichos
print(f"Valores reales y predichos para la variable '{variable_elegida}' para los primeros 5 datos:")
print(primeros_5_valores)


Valores reales y predichos para la variable 'Refrigerator' para los primeros 5 datos:
   Valor Real  Valor Predicho
0    0.544639        0.057759
1    0.309017        0.109593
2    0.284015        0.149718
3    0.000000        0.113913
4    0.284015        0.171309
5    0.983255        0.216332
6    0.078459        0.089559
7    0.104528        0.096761
8    0.207912        0.149811
9    0.000000        0.133240
